#Load the data and do pre-processing

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import pickle
def impute_missing_values(X):
    """
    For each feature column, impute missing values  (np.nan) with the 
    population mean for that feature.
    
    Args:
        X: np.array, shape (N, L, d). X could contain missing values
    Returns:
        X: np.array, shape (N, L, d). X does not contain any missing values
    """
    N, L, d = X.shape
    X = X.reshape(N*L, d)
    col_means = np.nanmean(X, axis=0)
    inds = np.where(np.isnan(X))
    X[inds] = np.take(col_means, inds[1])
    return X.reshape(N, L, d)
  
def standardize_features(X):
    """
    For each feature column, normalize all values to range [0, 1].
    Args:
        X: np.array, shape (N, d).
    Returns:
        X: np.array, shape (N, d). Values are normalized per column.
    """
    scaler = StandardScaler()
    N, L, d = X.shape
    X = X.reshape(N*L, d)
    X = scaler.fit_transform(X)
    return X.reshape(N, L, d)

#with open("features_labels_12000.p", 'rb') as f:
#      features, df_labels = pickle.load(f)
#      X = np.array([df_i.values for df_i in features])[0:10000,:,:]
#      y = df_labels['In-hospital_death'].values.copy()[0:10000]
#      y[y == -1] = 0

#np.save("X.npy",X)

X=np.load("X.npy")
y=np.load("y.npy")

X_train1,X_train2,X_test1,X_test2= X[0:8000,:,0:35],X[0:8000,:,35:70], X[8000:10000,:,0:35],X[8000:10000,:,35:70]
y_train,y_test=y[0:8000],y[8000:10000]

X_train1,X_test1=impute_missing_values(X_train1),impute_missing_values(X_test1)
X_train1,X_test1=standardize_features(X_train1),standardize_features(X_test1)
print(np.shape(X))


(10000, 48, 70)


# Defining layers

In [4]:
import random
print ("Random number with seed 0")
random.seed(0)
def transfer_y(y):
  size=np.shape(y)[0]
  result=np.zeros((size,2))
  for i in range(size):
    if y[i]==1:
      result[i,1]=1
    else:
      result[i,0]=1
  return result
tf.reset_default_graph() 
#[1, filter_width, in_channels, out_channels]
weights = {
    'wc1': tf.get_variable('W0', shape=(3,35,32), initializer=tf.contrib.layers.xavier_initializer()), 
    'wc2': tf.get_variable('W1', shape=(3,32,64), initializer=tf.contrib.layers.xavier_initializer()), 
    'wc3': tf.get_variable('W2', shape=(3,64,128), initializer=tf.contrib.layers.xavier_initializer()), 
    'wd1': tf.get_variable('W3', shape=(6*8*128,128), initializer=tf.contrib.layers.xavier_initializer()), 
    'out': tf.get_variable('W6', shape=(128,2), initializer=tf.contrib.layers.xavier_initializer()), 
}
biases = {
    'bc1': tf.get_variable('B0', shape=(32), initializer=tf.contrib.layers.xavier_initializer()),
    'bc2': tf.get_variable('B1', shape=(64), initializer=tf.contrib.layers.xavier_initializer()),
    'bc3': tf.get_variable('B2', shape=(128), initializer=tf.contrib.layers.xavier_initializer()),
    'bd1': tf.get_variable('B3', shape=(128),initializer=tf.contrib.layers.xavier_initializer()),
    'out': tf.get_variable('B4', shape=(2), initializer=tf.contrib.layers.xavier_initializer()),
}
training_iters = 200 
learning_rate = 0.001 
batch_size = 64
n_classes=2


x = tf.placeholder("float", [batch_size, 48, 35])   #input_length, dimension
x_missing = tf.placeholder("float", [batch_size, 48, 35])
y = tf.placeholder("float", [batch_size,2])

def conv1d(x, W, b,if_bias=True):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv1d(x, W, padding='SAME')
    if if_bias:
      x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x) 

def maxpool1d(x,k):
    return tf.nn.max_pool1d(x, ksize=k, strides=1,padding='SAME')



def conv_net(x,x_missing, weights, biases):  
    conv1 = conv1d(x, weights['wc1'], biases['bc1'])
    #originally, dimention of 48
    #adding an average/abs matrix
    conv1 = tf.nn.avg_pool1d(conv1,  ksize=2, strides=1,padding='SAME')
    conv2 = conv1d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 7*7 matrix.
    conv2 = maxpool1d(conv2, k=2)
    
    conv3 = conv1d(conv2, weights['wc3'], biases['bc3'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 4*4.
    conv3 = maxpool1d(conv3, k=2)
    #print(np.shape(conv3))
    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    #print(np.shape(fc1))
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    
    # Output, class prediction
    # finally we multiply the fully connected layer with the weights and add a bias term. 
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out
  
pred = conv_net(x,x_missing, weights, biases)
#print(np.shape(pred),np.shape(y))
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
with tf.Session() as sess:
    sess.run(init) 

    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    summary_writer = tf.summary.FileWriter('./Output', sess.graph)
    x_len=len(y_train)
    for i in range(training_iters):
        for batch in range(x_len//batch_size):
            batch_x = X_train1[batch*batch_size:min((batch+1)*batch_size,x_len),:,:]
            batch_x_missing = X_train2[batch*batch_size:min((batch+1)*batch_size,x_len),:,:]
            #print(np.shape(batch_x_missing))
            batch_y=transfer_y(y_train[batch*batch_size:min((batch+1)*batch_size,x_len)] )
            # Run optimization op (backprop).
                # Calculate batch loss and accuracy
            
            opt = sess.run(optimizer, feed_dict={x: batch_x,x_missing: batch_x_missing,\
                                                              y: batch_y})
            
            loss, auc = sess.run([cost, accuracy], feed_dict={x: batch_x,x_missing: batch_x_missing,\
                                                              y: batch_y})
        #print("Iter " + str(i) + ", Loss= " +"{:.6f}".format(loss) + ", Accuracy= " + "{:.5f}".format(auc))
        #print("Optimization Finished!")

        # Calculate accuracy for all 10000 mnist test images
        test_auc,valid_loss =0,0
        for k in range(25):
          from random import sample 
          index=np.array(range(64))+k*64
          temp1,temp2= sess.run([accuracy,cost], feed_dict={x: X_test1[index,:,:],x_missing: X_test2[index,:,:], y :transfer_y(y_train[index])})
          test_auc=test_auc+temp1
          valid_loss =valid_loss+temp2
        test_auc=test_auc/25
        valid_loss =valid_loss/25
        train_loss.append(loss)
        test_loss.append(valid_loss)
        train_accuracy.append(auc)
        test_accuracy.append(test_auc)
        if test_auc>test_accuracy[-1] and  test_auc>test_accuracy[-2]:
          print("Testing Accuracy:","{:.5f}".format(test_auc))
    summary_writer.close()

Random number with seed 0


W0712 20:01:38.389577 140363388868480 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0712 20:01:38.511859 140363388868480 deprecation.py:323] From <ipython-input-4-543b079efd9e>:78: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



# training the data

tutorial:
https://www.datacamp.com/community/tutorials/cnn-tensorflow-python

costumized layer

https://www.tensorflow.org/tutorials/eager/custom_layers

reliability based on missingness and distance based on time.



In [11]:
import random
print ("Random number with seed 0")
random.seed(0)
def transfer_y(y):
  size=np.shape(y)[0]
  result=np.zeros((size,2))
  for i in range(size):
    if y[i]==1:
      result[i,1]=1
    else:
      result[i,0]=1
  return result
tf.reset_default_graph() 
#[1, filter_width, in_channels, out_channels]
weights = {
    'wc1': tf.get_variable('W0', shape=(3,35,32), initializer=tf.contrib.layers.xavier_initializer()), 
    'wc2': tf.get_variable('W1', shape=(3,32,64), initializer=tf.contrib.layers.xavier_initializer()), 
    'wc3': tf.get_variable('W2', shape=(3,64,128), initializer=tf.contrib.layers.xavier_initializer()), 
    'wd1': tf.get_variable('W3', shape=(6*8*128,128), initializer=tf.contrib.layers.xavier_initializer()), 
    'out': tf.get_variable('W6', shape=(128,2), initializer=tf.contrib.layers.xavier_initializer()), 
}
biases = {
    'bc1': tf.get_variable('B0', shape=(32), initializer=tf.contrib.layers.xavier_initializer()),
    'bc2': tf.get_variable('B1', shape=(64), initializer=tf.contrib.layers.xavier_initializer()),
    'bc3': tf.get_variable('B2', shape=(128), initializer=tf.contrib.layers.xavier_initializer()),
    'bd1': tf.get_variable('B3', shape=(128),initializer=tf.contrib.layers.xavier_initializer()),
    'out': tf.get_variable('B4', shape=(2), initializer=tf.contrib.layers.xavier_initializer()),
}
training_iters = 200 
learning_rate = 0.001 
batch_size = 64
n_classes=2


x = tf.placeholder("float", [batch_size, 48, 35])   #input_length, dimension
x_missing = tf.placeholder("float", [batch_size, 48, 35])
y = tf.placeholder("float", [batch_size,2])

def conv1d(x, W, b,if_bias=True):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv1d(x, W, padding='SAME')
    if if_bias:
      x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x) 

def maxpool1d(x,k):
    return tf.nn.max_pool1d(x, ksize=k, strides=1,padding='SAME')



def conv_net(x,x_missing, weights, biases):  
    conv1 = conv1d(x, weights['wc1'], biases['bc1'])
    #originally, dimention of 48
    #adding an average/abs matrix
    conv_missing=conv1d(x_missing,weights['wc1'],biases['bc1'],False)
    
    w=1-tf.math.abs(tf.math.l2_normalize(conv_missing))
    conv1 = tf.nn.avg_pool1d(tf.math.multiply(conv1,w),  ksize=2, strides=1,padding='SAME')
   
    conv2 = conv1d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 7*7 matrix.
    conv2 = maxpool1d(conv2, k=2)
    
    conv3 = conv1d(conv2, weights['wc3'], biases['bc3'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 4*4.
    conv3 = maxpool1d(conv3, k=2)
    #print(np.shape(conv3))
    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    #print(np.shape(fc1))
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    
    # Output, class prediction
    # finally we multiply the fully connected layer with the weights and add a bias term. 
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out
  
pred = conv_net(x,x_missing, weights, biases)
#print(np.shape(pred),np.shape(y))
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
with tf.Session() as sess:
    sess.run(init) 

    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    summary_writer = tf.summary.FileWriter('./Output', sess.graph)
    x_len=len(y_train)
    for i in range(training_iters):
        for batch in range(x_len//batch_size):
            batch_x = X_train1[batch*batch_size:min((batch+1)*batch_size,x_len),:,:]
            batch_x_missing = X_train2[batch*batch_size:min((batch+1)*batch_size,x_len),:,:]
            #print(np.shape(batch_x_missing))
            batch_y=transfer_y(y_train[batch*batch_size:min((batch+1)*batch_size,x_len)] )
            # Run optimization op (backprop).
                # Calculate batch loss and accuracy
            
            opt = sess.run(optimizer, feed_dict={x: batch_x,x_missing: batch_x_missing,\
                                                              y: batch_y})
            
            loss, auc = sess.run([cost, accuracy], feed_dict={x: batch_x,x_missing: batch_x_missing,\
                                                              y: batch_y})
        #print("Iter " + str(i) + ", Loss= " +"{:.6f}".format(loss) + ", Accuracy= " + "{:.5f}".format(auc))
        #print("Optimization Finished!")

        # Calculate accuracy for all 10000 mnist test images
        test_auc,valid_loss =0,0
        for k in range(25):
          from random import sample 
          index=np.array(range(64))+k*64
          temp1,temp2= sess.run([accuracy,cost], feed_dict={x: X_test1[index,:,:],x_missing: X_test2[index,:,:], y :transfer_y(y_train[index])})
          test_auc=test_auc+temp1
          valid_loss =valid_loss+temp2
        test_auc=test_auc/25
        valid_loss =valid_loss/25
        train_loss.append(loss)
        test_loss.append(valid_loss)
        train_accuracy.append(auc)
        test_accuracy.append(test_auc)
        if i >3:
          if test_auc<test_accuracy[-3] and  test_auc<test_accuracy[-2]:
            print("Testing Accuracy:","{:.5f}".format(max(test_accuracy)))
            break
    summary_writer.close()
          

Random number with seed 0
0.75625 0.756875
Testing Accuracy: 0.80812


In [0]:
from numpy.linalg import norm
X=np.zeros((2,2,3))
X[1,:,:]=[[2,-5,8],[4,2,0]]
X[0,:,:]=[[2,5,1],[4,2,8]]

def norm_to_1(X):
  w=tf.Session().run(X)
  size=np.shape(X)[2]
  for i in range(size):
    summ=np.sum(X[:,:,i])
    if not summ==0:
       X[:,:,i]=X[:,:,i]/summ
  return X


In [0]:
y=np.load("y.npy")

X_train1,X_train2,X_test1,X_test2= X[0:8000,:,0:35],X[0:8000,:,35:70], X[8000:10000,:,0:35],X[8000:10000,:,35:70]
y_train,y_test=y[0:8000],y[8000:10000]

X_train1,X_test1=impute_missing_values(X_train1),impute_missing_values(X_test1)
X_train1,X_test1=standardize_features(X_train1),standardize_features(X_test1)
print(np.shape(X))

(10000, 48, 70)
